# install and load dependancy

In [ ]:
!pip install transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.2 MB/s eta 0:00:00


In [ ]:
import random
import time
import csv
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

import torch
from torch.utils.data import Dataset, TensorDataset,DataLoader,RandomSampler,SequentialSampler

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

from transformers import BertTokenizer as bertTokenizer
from transformers import BertForSequenceClassification as bfsc,AdamW,BertConfig
from transformers import get_linear_schedule_with_warmup

## Link with Google Drive

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

# path
%cd "/content/drive/MyDrive/Telecom/Airbus/Offenseval/Training/"

Mounted at /content/drive
/content/drive/MyDrive/Telecom/Airbus/Offenseval/Training


## Cuda Setup

In [ ]:
gpuname=""
device=""
y=""
preprocessedTweets=""
ids_of_sentence=[]
ids_of_sentence_words=[]


In [ ]:
gpuname=tf.test.gpu_device_name()
if gpuname=='/device:GPU:0':
  print('Found GPU at :{}'.format(gpuname))
else:
  gpuname=""
if torch.cuda.is_available():
  device=torch.device("cuda")
  n_gpu=torch.cuda.device_count()
  print("The device name is %s"%torch.cuda.get_device_name(0))
else:
  print("No GPU available using only CPU instead")
  device=torch.device("cpu")

Found GPU at :/device:GPU:0
The device name is Tesla T4


# Load data

In [ ]:
headers=['id','tweet','subtask_a','subtask_b','subtask_c']
englishdata = pd.read_csv("olid-training-v1.0.tsv", delimiter='\t',names=headers,low_memory=False)
englishdata=englishdata[['id','tweet','subtask_a']]
englishdata=englishdata[1:]

In [ ]:
englishdata

,id,tweet,subtask_a
1,86426,@USER She should ask a few native Americans wh...,OFF
2,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF
3,16820,Amazon is investigating Chinese employees who ...,NOT
4,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF
5,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT
...,...,...,...
13236,95338,@USER Sometimes I get strong vibes from people...,OFF
13237,67210,Benidorm ✅ Creamfields ✅ Maga ✅ Not too sh...,NOT
13238,82921,@USER And why report this garbage. We don't g...,OFF
13239,27429,@USER Pussy,OFF


In [ ]:
label_counts = englishdata['subtask_a'].value_counts()

off_count = label_counts.get('OFF', 0)
not_count = label_counts.get('NOT', 0)

print("Number of OFF labels:", off_count)
print("Number of NOT labels:", not_count)

Number of OFF labels: 4400
Number of NOT labels: 8840


# Pre-processing

In [ ]:
dfnumpy=englishdata.to_numpy()
x_train=dfnumpy[:int(len(dfnumpy)*0.8), 1].reshape(-1, 1)
y_train=dfnumpy[:int(len(dfnumpy)*0.8), 2].reshape(-1, 1)

In [ ]:
x_train

array([['@USER She should ask a few native Americans what their take on this is.'],
       ['@USER @USER Go home you’re drunk!!! @USER #MAGA #Trump2020 👊🇺🇸👊 URL'],
       ['Amazon is investigating Chinese employees who are selling internal data to third-party sellers looking for an edge in the competitive marketplace. URL #Amazon #MAGA #KAG #CHINA #TCOT'],
       ...,
       ['@USER I have been put on an Antifa “hit list” and i am taking the matter very seriously. I have reported to Twitter- nothing. Group: Targets for (maybe of) Antifa.  Donald Trump and Candace Owens as well. All I have is the Twitter handle. Thanks 🙏🏻 URL'],
       ['@USER Luckily you’re in one of the safest @USER seats. It allows you to be flippant/dismissive when it comes to opposing views. Still not heard a cohesive alternative plan from you though... #brexitshambles'],
       ['@USER Congratulations! He is so adorable']], dtype=object)

In [ ]:
dfnumpy=englishdata.to_numpy()
x_test=dfnumpy[int(len(dfnumpy)*0.8):, 1].reshape(-1, 1)
y_test=dfnumpy[int(len(dfnumpy)*0.8):, 2].reshape(-1, 1)

In [ ]:
x_test

array([['@USER Nancy Lee Grahn You Are Awesome! I have been a fan since Santa Barbara!! Alex Davis also Rocks!!!!! Thank you !!!'],
       ['@USER She is a Skrull. Enemy of The Kree. The Kree are who gave Carol her powers and whose uniform she is wearing in the first few moments of the trailer.'],
       ['@USER @USER @USER @USER @USER @USER @USER Except you kind of are when it comes to gun control'],
       ...,
       ["@USER And why report this garbage.  We don't give a crap."],
       ['@USER Pussy'],
       ['#Spanishrevenge vs. #justice #HumanRights and #FreedomOfExpression #Spain is a  #fakedemocracy @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER #cddr #shameonSpain #WakeupEurope @USER URL']],
      dtype=object)

In [ ]:
le = preprocessing.LabelEncoder()
yTrain=le.fit_transform(y_train)
yTest=le.fit_transform(y_test.flatten())

(10592, 1)
(2648, 1)
(10592,)
['NOT' 'OFF']
['NOT' 'OFF']


/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## Load the data into the dataloader

In [ ]:
class EnglishTrainDataset(Dataset):
    def __init__(self,xytrain):
        self.xytrain = xytrain
        self.maxlength=MAXLENGTH

    def __getitem__(self, index):
        tokenized_review = tokenizer.tokenize(str(self.xytrain[0][index].flatten()))
        if len(tokenized_review) > self.maxlength:
            tokenized_review = tokenized_review[:self.maxlength]
        ids_of_sentence_word  = tokenizer.convert_tokens_to_ids(tokenized_review)
        padding = [0] * (self.maxlength - len(ids_of_sentence_word))
        ids_of_sentence_word += padding
        assert len(ids_of_sentence_word) == self.maxlength
        attention_mask = [int(b > 0) for b in ids_of_sentence_word]
        x_train_pytorch = torch.tensor(ids_of_sentence_word)
        y_train_pytorch=torch.tensor(self.xytrain[1][index])
        x_train_mask_pytorch=torch.tensor(attention_mask)
        return x_train_pytorch,x_train_mask_pytorch,y_train_pytorch

    def __len__(self):
        return len(self.xytrain[0])

In [ ]:
MAXLENGTH=64
xytrain=[x_train,yTrain]
tokenizer=bertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
tdataset = EnglishTrainDataset(xytrain)
tsampler=RandomSampler(tdataset)
tdataloader = DataLoader(tdataset, batch_size=32, num_workers=1, shuffle=False,sampler=tsampler)

In [ ]:
class EnglishTestDataset(Dataset):
    def __init__(self,xytest):
        self.xytest = xytest
        self.maxlength=MAXLENGTH

    def __getitem__(self, index):
        tokenized_review = tokenizer.tokenize(str(self.xytest[0][index].flatten()))
        if len(tokenized_review) > self.maxlength:
            tokenized_review = tokenized_review[:self.maxlength]
        ids_of_sentence_word  = tokenizer.convert_tokens_to_ids(tokenized_review)
        padding = [0] * (self.maxlength - len(ids_of_sentence_word))
        ids_of_sentence_word += padding
        assert len(ids_of_sentence_word) == self.maxlength
        attention_mask = [int(b > 0) for b in ids_of_sentence_word]
        x_test_pytorch = torch.tensor(ids_of_sentence_word)
        y_test_pytorch=torch.tensor(self.xytest[1][index])
        x_test_mask_pytorch=torch.tensor(attention_mask)
        return x_test_pytorch,x_test_mask_pytorch,y_test_pytorch

    def __len__(self):
        return len(self.xytest[0])

In [ ]:
xytest=[x_test,yTest]
tokenizer=bertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
tedataset = EnglishTestDataset(xytest)
tesampler=RandomSampler(tedataset)
tedataloader = DataLoader(tedataset, batch_size=32, num_workers=1, shuffle=False,sampler=tesampler)

# Load model

In [ ]:
model=bfsc.from_pretrained('bert-base-uncased',num_labels=2,output_attentions=False,output_hidden_states=False)
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
##ONLY FOR THE FIRST TIME SAVE ELSE LOAD
torch.save({'state_dict': model.state_dict()}, 'model_offenseEval_taskA.pth.tar')
checkpoint = torch.load('model_offenseEval_taskA.pth.tar')
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

# Setup training parameters

In [ ]:
params=list(model.named_parameters())

In [ ]:
no_decay = ["bias", "beta","LayerNorm.weight","gamma"]
optimizer_grouped_parameters = [
{
"params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
"weight_decay": 0.01,
},
{"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
]

In [ ]:
optimizer=AdamW(model.parameters(),lr=2e-5,eps=1e-8)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
def calculateF1Score(predictions,labels):
  predictionArgmax=np.argmax(predictions,axis=1).flatten()
  labelsFlattend=labels.flatten()
  return f1_score(labelsFlattend, predictionArgmax, average='macro'),accuracy_score(labelsFlattend, predictionArgmax)

# Start the training

In [ ]:
epochs=30
total_steps=len(tdataloader)*epochs
sch=get_linear_schedule_with_warmup(optimizer,
                                    num_warmup_steps=0,num_training_steps=total_steps)

In [ ]:
def set_seed(seed,ngpu):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if ngpu > 0:
        torch.cuda.manual_seed_all(seed)

In [ ]:
set_seed(42,torch.cuda.device_count())

lossList=[]
max_grad_norm=1.0
for e in range(0, epochs):

    print("Start Epoch Number",(e + 1))
    print("Start Training")
    if device.type=="cpu":
     model.to(device)
     map_location='cpu'
    else:
      model.cuda()
      map_location=lambda storage, loc: storage.cuda()
    checkpoint = torch.load('bertenglish1.pth.tar',map_location=map_location)
    model.load_state_dict(checkpoint['state_dict'])

    t1 = time.time()
    tr_loss, logging_loss = 0.0, 0.0
    model.train()
    tsteps=0
    for step, batch in enumerate(tdataloader):
        if step % 50 == 0 and not step == 0:
            print("Batch Completed  {:,}  of  {:,}.    Elapsed time is  {}".format(step, len(tdataloader),time.time() - t1))
        batch = tuple(t.to(device) for t in batch)
        inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[2]}
        model.zero_grad()
        outputs = model(inputs["input_ids"],token_type_ids=None,attention_mask=inputs["attention_mask"], labels=inputs["labels"])
        loss = outputs[0]
        loss.backward()
        tr_loss += loss.item()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        tsteps+=1
        optimizer.step()
        sch.step()
    a_tr_loss = tr_loss /(tsteps)
    lossList.append(a_tr_loss)
    print(" The training loss incured is  {0:.3f}".format(a_tr_loss))
    t2=time.time()
    print("  Training one epoch time taken",t2-t1)
    print(" Validation starts here ")
    t1 = time.time()
    model.eval()
    eval_loss = 0
    nb_eval_steps = 0
    eval_f1=0
    eval_acc=0

    for batch_idx, data in enumerate(tedataloader):

        batch = tuple(t.to(device) for t in data)
        inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[2]}
        with torch.no_grad():
           outputs = model(inputs["input_ids"],token_type_ids=None,attention_mask=inputs["attention_mask"])
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = (inputs["labels"]).to('cpu').numpy()
        tmpf1score,tmpaccscore = calculateF1Score(logits, label_ids)
        eval_f1 = eval_f1+tmpf1score
        eval_acc=eval_acc+tmpaccscore
        nb_eval_steps += 1
    torch.save({'state_dict': model.state_dict()}, 'model_offenseEval_taskA.pth.tar')
    print("  F1 score: {0:.3f}".format(eval_f1/nb_eval_steps))
    print("  Accuracy score: {0:.3f}".format(eval_acc/nb_eval_steps))
    t2=time.time()
    print("  Validating one epoch time taken ",t2-t1)

print("ALL DONE!!!")

Start Epoch Number 1
Start Training
Batch Completed  50  of  331.    Elapsed time is  22.578232049942017
Batch Completed  100  of  331.    Elapsed time is  41.00574803352356
Batch Completed  150  of  331.    Elapsed time is  58.75213861465454
Batch Completed  200  of  331.    Elapsed time is  76.37273335456848
Batch Completed  250  of  331.    Elapsed time is  94.2712516784668
Batch Completed  300  of  331.    Elapsed time is  112.81922793388367
 The training loss incured is  0.512
  Training one epoch time taken 124.15308403968811
 Validation starts here 
  F1 score: 0.767
  Accuracy score: 0.809
  Validating one epoch time taken  11.929500341415405
Start Epoch Number 2
Start Training
Batch Completed  50  of  331.    Elapsed time is  18.06206464767456
Batch Completed  100  of  331.    Elapsed time is  35.99024510383606
Batch Completed  150  of  331.    Elapsed time is  54.02660250663757
Batch Completed  200  of  331.    Elapsed time is  72.04335045814514
Batch Completed  250  of  331.

KeyboardInterrupt: ignored